# Access CCMP data on Pangeo

This tutorial shows how to access the CCMP wind speed data, makes a few plots of the data, and then compares it to a buoy wind speed.

# CCMP Winds in a cloud-optimized-format for Pangeo

The Cross-Calibrated Multi-Platform (CCMP) Ocean Surface Wind Vector Analyses is part of the NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) Program. MEaSUREs, develops consistent global- and continental-scale Earth System Data Records by supporting projects that produce data using proven algorithms and input.  If you use this data, please give [credit](https://podaac.jpl.nasa.gov/MEaSUREs-CCMP?sections=about).  For more information, please review the [documentation](https://podaac-tools.jpl.nasa.gov/drive/files/allData/ccmp/L2.5/docs/ccmp_users_guide.pdf). Please note that this data is not recommended for trend calculations.

Data is NRT from 4/1/2019 - 10/4/2020.
The 6-hourly VAM analyses are centered at 0,6,12 and 18z

# Accessing cloud satellite data

- CCMP zarr conversion funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program and AWS Public Dataset Program
  
## Credit:
- [Chelle Gentemann](mailto:cgentemann@faralloninstitute.org), [Farallon Institute](http://www.faralloninstitute.org/), [Twitter](https://twitter.com/ChelleGentemann) - creation of Zarr data store and tutorial
- [Charles Blackmon Luca](mailto:blackmon@ldeo.columbia.edu), [LDEO](https://www.ldeo.columbia.edu/) - help with moving to Pangeo storage and intake update
- [Willi Rath](mailto:wrath@geomar.de), [GEOMAR](https://www.geomar.de/en/), [Twitter](https://twitter.com/RathWilli) - motivated CG to move data to Pangeo!
- [Tom Augspurger](mailto:taugspurger@microsoft.com), [Microsoft](https://www.linkedin.com/in/tom-augspurger-01269315/),[Twitter](https://twitter.com/TomAugspurger) - help updating the data.

### Zarr data format

 [Zarr](https://zarr.readthedocs.io/en/stable/)

### Data proximate computing
These are BIG datasets that you can analyze on the cloud without downloading the data. You can run this on your phone, a Raspberry Pi, laptop, or desktop.   
By using public cloud data, your science is reproducible and easily shared!

### To run this notebook

Code is in the cells that have <span style="color: blue;">In [  ]:</span> to the left of the cell and have a colored background

To run the code:
- option 1) click anywhere in the cell, then hold `shift` down and press `Enter`
- option 2) click on the Run button at the top of the page in the dashboard

Remember:
- to insert a new cell below press `Esc` then `b`
- to delete a cell press `Esc` then `dd`

### First start by importing libraries

In [ ]:
#libs for reading data
import xarray as xr
import intake
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import cartopy.crs as ccrs

#lib for dask gateway
from dask_gateway import Gateway
from dask.distributed import Client
from dask import delayed

### Start a cluster, a group of computers that will work together.

(A cluster is the key to big data analysis on on Cloud.)

- This will set up a [dask kubernetes](https://docs.dask.org/en/latest/setup/kubernetes.html) cluster for your analysis and give you a path that you can paste into the top of the Dask dashboard to visualize parts of your cluster.  
- You don't need to paste the link below into the Dask dashboard for this to work, but it will help you visualize progress.
- Try 20 workers to start (during the tutorial) but you can increase to speed things up later

In [ ]:
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=75)
client = Client(cluster)
cluster

** ☝️ Don’t forget to click the link above or copy it to the Dask dashboard (orange icon) on the left to view the scheduler dashboard! **

### Initialize Dataset

Here we load the dataset from the zarr store. Note that this very large dataset (273 GB) initializes nearly instantly, and we can see the full list of variables and coordinates.

### Examine Metadata

For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent
Printing the dataset will show you the dimensions, coordinates, and data variables with clickable icons at the end that show more metadata and size.

In [ ]:
%%time
cat_pangeo = intake.open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml")

ds_ccmp = cat_pangeo.atmosphere.nasa_ccmp_wind_vectors.to_dask()

#calculate wind speed and add attributes to new variable
ds_ccmp['wspd'] = np.sqrt(ds_ccmp.uwnd**2 + ds_ccmp.vwnd**2)
ds_ccmp.wspd.attrs=ds_ccmp.uwnd.attrs
ds_ccmp.wspd.attrs['long_name']='wind speed at 10 meters'

ds_ccmp

# Plot a global image of the data on 7/4/2020 (randomly chosen day)

``xarray`` makes plotting the data very easy.  A nice overview of plotting with xarray is [here](http://xarray.pydata.org/en/stable/plotting.html).  Details on [.plot](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.plot.html#xarray.DataArray.plot)

Below we are plotting the variable `wspd` (wind speed) and `nobs` (number of observations) to show how there is data over the ocean but none over land. CCMP data includes 25 km ERA5 data over land.

In [ ]:
%%time

day = ds_ccmp.sel(time='2020-09-08T18')

fig, ax = plt.subplots(1,2, figsize=(18,6))

day.wspd.plot(ax=ax[0])

day.nobs.plot(ax=ax[1])

In [ ]:
import cartopy
import cartopy.crs as ccrs
ortho = ccrs.Orthographic(-110, 20)           # define target coordinate frame
geo = ccrs.PlateCarree()                     # define origin coordinate frame

plt.figure(figsize=(7,5))                    #set the figure size
ax = plt.subplot(1, 1, 1, projection=ortho)  #create the axis for plotting
q = day.uwnd.plot(ax=ax, 
                            transform = geo, 
                            cmap='RdBu_r', #'cmo.thermal', 
                            vmin=-5, 
                            vmax=5,cbar_kwargs={'label':'Eastward Wind Speed (m/s)'}) # plot a colormap in transformed coordinates cmap='OrRd', 
global_extent = ax.get_extent(crs=ccrs.PlateCarree())
#ax.add_feature(cartopy.feature.LAND,color='gray')
ax.add_feature(cartopy.feature.COASTLINE)

plt.savefig('./../../figures/ccmp_wind_example.png')


# Plot a timeseries of the average wind speed over a region

- use [.sel](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html#xarray-dataarray-sel) and slice to select a region of data
- use [.mean](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.mean.html#xarray.DataArray.mean) to calculate the spatial mean
- use [.plot](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.plot.html#xarray-dataarray-plot) to plot a timeseries of the mean wind speed

In [ ]:
%%time

subset = ds_ccmp.sel(latitude=slice(30,50),longitude=slice(200,210))

ts = subset.mean({'latitude','longitude'},keep_attrs=True)

ts.wspd.plot()

# Plot a map of the annual average wind speed

In [ ]:
%%time

ds_ccmp.sel(time=slice('2000-01-01','2000-12-31')).mean({'time'}).wspd.plot()

# Make a Hovmoller type plot

In [ ]:
%%time

ds_ccmp.sel(latitude=0.125,longitude=slice(120,275)).wspd.plot(vmin=0,vmax=12,cmap='magma')

## In Feb 2020 a GRL [paper](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL091430) came out connecting 3 closely occuring Typhoons near Korea to the California wildfires
  
"Strong winds that accentuated a fire outbreak in the western United States in early September of 2020 resulted from an atmospheric wave train that spanned the Pacific Ocean. Days before the atmospheric waves developed in the United States, three western Pacific tropical cyclones (typhoons) underwent an extratropical transition over Korea within an unprecedentedly short span of 12 days. "

Using ERA5 winds, Figure 1 showed zonal winds averaged over a box located over NCal/Oregon. On 9/8 and again on 9/24 the zonal winds were strongly negative (Diablo winds) and both events were associated with major fires.

Below, let's do the same figure with CCMP winds - Note that CCMP winds end 10/4/2020, so the data will end then 

In [ ]:
ts = ds_ccmp.uwnd.sel(longitude=slice(236,239),latitude=slice(41,46)).mean({'latitude','longitude'}).load()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
for lyr in range(1979,2021):
    yr = ts.sel(time=slice(str(lyr)+'-08-01',str(lyr)+'-10-29'))
    xx=yr.time.dt.dayofyear+yr.time.dt.hour/24
    ax.plot(xx,yr,color='silver')
    if lyr==2020:
        ax.plot(xx,yr,color='red')
date_form = DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)

# Compare to buoy data
Data from NDBC buoys, which measure wind speed are [here](https://dods.ndbc.noaa.gov/) and can be read via an THREDDS server.
- read in NDBC buoy data
- find closest CCMP data and linearly interpolate to buoy location
- examine a timeseries, caluclate bias and STD

In [ ]:
url='https://dods.ndbc.noaa.gov/thredds/dodsC/data/cwind/51003/51003.ncml'

with xr.open_dataset(url) as ds:
    # The longitude is on a -180 to 180, CCMP is 0-360, so make sure to convert
    ds.coords['longitude'] = np.mod(ds['longitude'], 360)
    
ds

In [ ]:
time_start = np.max([ds.time[0].data,ds_ccmp.time[0].data])
time_stop = np.min([ds.time[-1].data,ds_ccmp.time[-1].data])

#cut off time before / after ccmp obs, linearly interpol data for buoy location
ccmp_buoy = ds_ccmp.sel(time=slice(time_start,time_stop)).interp(latitude=ds.latitude,longitude=ds.longitude,method='linear')

#cut off time before / after ccmp obs
ds = ds.sel(time=slice(time_start,time_stop))

In [ ]:
%%time

# go from 30 min to 6-hourly sampling, resample with mean for +- 3 hours centered on timestamp
#ds = ds.resample(time='6H',loffset='180min',base=3).mean()  
# OMG!!! resample is sooo slow.  doing this cludge instead, same to within 6 significant digits

ds = ds.rolling(time=36,center=True).mean() 
ds_buoy = ds.interp(time=ccmp_buoy.time,method='nearest')

ds_buoy

In [ ]:
ccmp_buoy.wspd.plot(color='r')
ds_buoy.wind_spd.plot(color='b')

In [ ]:
%%time
# let's remove the seasonal cycle so we can compare the data more clearly

#for ccmp data create the climatology and anomaly
ts_ccmp_climatology = ccmp_buoy.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=True)
ts_ccmp_anomaly = (ccmp_buoy.groupby('time.dayofyear')-ts_ccmp_climatology).compute()

#for buoy data create the climatology and anomaly
ts_buoy_climatology = ds_buoy.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=True)
ts_buoy_anomaly = (ds_buoy.groupby('time.dayofyear')-ts_buoy_climatology).compute()

In [ ]:
(ts_ccmp_anomaly.wspd-ts_buoy_anomaly.wind_spd).plot()

In [ ]:
print('buoy minus satellite wind speeds')
rdif = (ts_buoy_anomaly.wind_spd-ts_ccmp_anomaly.wspd)
print('mean:',rdif.mean().data)
print('std:',rdif.std().data)

# Masking data and looking at frequency of low/high wind speeds.
- Make a land/ocean/ice mask to show where there is actually data

### Three different ways to mask the data
1. A daily mask that removes data with sea ice and land
- sum over time for nobs (number of observations) variable
- average over a month so that land and monthly sea ice are masked out
2. A mask that removes all data that over land or where there is 'permanent' sea ice
- find when nobs is > 0
3. A climatology mask that removes all data that over land or where there has ever been sea ice
- sum over time for nobs (number of observations) variable
- average over a month so that land and monthly sea ice are masked out

# Apply the mask 
- over land, CCMP is ERA5 data
- for many ocean applications a land / sea ice mask is needed
- below are some different mask options that use the CCMP data to generate a mask


In [ ]:
def mask_data(ds,type):
    if type=='daily': #daily mask removes sea ice and land
        mask_obs = ds.nobs.rolling(time=180,center=True).max('time')  #4 per day 30 days = 180 rolling window
        cutoff = 0
    if type=='land':  # land mask only (includes data over sea ice)
        mask_obs = ds.nobs.sum({'time'},keep_attrs=True)  #this will give you a LAND mask
        cutoff = 0
    if type=='climatology':  #climatology mask removes max sea ice extent and land
        mask_obs = ds.nobs.rolling(time=180,center=True).max('time')  #4 per day 30 days = 180 rolling window
        mask_obs = mask_obs.sum({'time'},keep_attrs=True)
        cutoff = 125000
    dy_mask = mask_obs>cutoff
    dy_mask = dy_mask.compute() #computing the mask speeds up subsequent operations
    masked = ds.where(dy_mask)
    return masked,dy_mask

- For this tutorial we will use the climatology mask

- This code will take ~10 min to run through the entire dataset to create the climatology mask

In [ ]:
%%time

masked,mask_obs = mask_data(ds_ccmp,'climatology')

mask_obs.plot()

In [ ]:
%%time

fig, ax = plt.subplots(1,3, figsize=(18,6))

masked.wspd[100,:,:].plot(ax=ax[0])

masked.wspd[-100,:,:].plot(ax=ax[1])

masked.wspd[5000,:,:].plot(ax=ax[2])

# calculate percentage of low/ave/high winds at each location

# a spatial map showing a climatology of roughness.  


In [ ]:
%%time

#Ideally in 3 panels - (a) at Hs=<2, (b) at Hs=mean wind speed (c) Hs> 10 

wnd = ds_ccmp.wspd.where(ds_ccmp.wspd<=2)
f2 = ((wnd/wnd).sum({'time'})/len(wnd.time)*100).compute()  # percent less than or equal to 2 m/s

wnd = ds_ccmp.wspd.where(ds_ccmp.wspd>10)
f10 = ((wnd/wnd).sum({'time'})/len(wnd.time)*100).compute()  # percent >= 10 m/s

ff = xr.concat([f2,f10],dim='frac')

In [ ]:
plt.rcParams['figure.figsize'] = (8.0,4.0)
plt.rcParams.update({'font.size': 12})
fg = ff.plot(vmin=0, vmax=100,
    col="frac",
    transform=ccrs.PlateCarree(),  # remember to provide this!
    subplot_kws={
        "projection": ccrs.PlateCarree()
    },
    cbar_kwargs={"label":'Percent',"orientation": "horizontal", "shrink": 0.8, "aspect": 40},
    robust=True,
)
tstr = ['< 2 m/s','> 10 m/s']
for i, ax in enumerate(fg.axes.flat):
    ax.set_title(tstr[i]) 
fg.map(lambda: plt.gca().coastlines())

# ocean only

In [ ]:
masked = ff.where(mask_obs>0)

fg = masked.plot(vmin=0, vmax=100,
    col="frac",
    transform=ccrs.PlateCarree(),  # remember to provide this!
    subplot_kws={
        "projection": ccrs.PlateCarree()
    },
    cbar_kwargs={"label":'Percent',"orientation": "horizontal", "shrink": 0.8, "aspect": 40},
    robust=True,
)
tstr = ['< 2 m/s','> 10 m/s']
for i, ax in enumerate(fg.axes.flat):
    ax.set_title(tstr[i]) 
fg.map(lambda: plt.gca().coastlines())


In [ ]:
cluster.close()